# Подготовка среды

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/Shared drives/hse_BERT/hse_Af_Tr_BERT')

In [3]:
folder = "bert-base-multilingual-cased-polit"

if not os.path.isdir(os.path.join("embeddings", folder)):
    os.mkdir(os.path.join("embeddings", folder))
if not os.path.isdir(os.path.join("embeddings", folder, "bin")):
    os.mkdir(os.path.join("embeddings", folder, "bin"))
if not os.path.isdir(os.path.join("embeddings", folder, "ready")):
    os.mkdir(os.path.join("embeddings", folder, "ready"))

In [4]:
!pip install transformers
!pip install razdel

     |████████████████████████████████| 2.1MB 5.4MB/s 
     |████████████████████████████████| 901kB 19.8MB/s 
     |████████████████████████████████| 3.3MB 33.0MB/s 


In [5]:
import torch
import nltk
import re
import json
import time
import numpy as np
import pickle as pc
from transformers import BertTokenizer, BertModel

import matplotlib.pyplot as plt
% matplotlib inline

from scipy.spatial.distance import cosine
from razdel import sentenize

In [6]:
# Load pre-trained model tokenizer (vocabulary)
# 'DeepPavlov/rubert-base-cased-sentence'
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

In [7]:
model = BertModel.from_pretrained('bert-base-multilingual-cased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
         

In [8]:
def my_dump(file, name):
    with open("./bin/" + name + ".pic", 'wb') as f:
        pc.dump(file, f)

def my_load(name):
    with open("./bin/" + name + ".pic", 'rb') as f:
        return pc.load(f)

#text processing

## generate (split) sentences

In [ ]:
%%time
# name = "all_cyberleninka_polit2"
name = "all_cyberleninka_liter2"
with open("./texts/{}/{}.txt".format(name, name), 'r') as f:
    data = f.read()
sent2 = list(sentenize(data))
my_dump(sent2, name)

CPU times: user 6min 1s, sys: 12.6 s, total: 6min 14s
Wall time: 6min 48s


#getting embeddings

## read sentences

In [9]:
%%time
name = "all_cyberleninka_polit2"
# name = "all_cyberleninka_liter2"
sent1 = my_load(name)

CPU times: user 9.24 s, sys: 6.06 s, total: 15.3 s
Wall time: 23.8 s


In [10]:
def get_text_from_ST(ss):
    return ss.text
sent = list(map(get_text_from_ST, sent1))
len(sent)

2348859

##process

In [11]:
def sum_tensors(token):
    return torch.sum(token[-4:], dim=0)

def get_embeddings(text):
    tokenized_text = tokenizer.tokenize("[CLS] " + text.lower() + " [SEP]")
    if len(tokenized_text) > 80:
        return [],[]
    marked_text = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)

    tokens_tensor = torch.tensor([marked_text])
    segments_tensors = torch.tensor([segments_ids])

    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]

    token_embeddings = torch.squeeze(torch.stack(hidden_states, dim=0),
                                    dim=1).permute(1,0,2)
    
    token_vecs_sum = list(map(sum_tensors, token_embeddings))
    return tokenized_text, token_vecs_sum

# token_vecs = hidden_states[-2][0]
# sentence_embedding = torch.mean(token_vecs, dim=0)

In [ ]:
%%time
batch = 1000
start_time = time.time()

ftom_b = 677
to_b = 850

print("senteces from {} to {}".format(ftom_b * batch, to_b * batch))

for j in range(ftom_b, to_b):
    pr_tr = {}
    temp_time = time.time()
    for i in range(j * batch, (j + 1) * batch):
        tt, tvs = get_embeddings(sent[i])
        for i in range(len(tt)):
            if tt[i].isalpha() and len(tt[i]) > 2 :
                if tt[i] in pr_tr:
                    pr_tr[tt[i]].append(tvs[i])
                else:
                    pr_tr[tt[i]] = [tvs[i]]

    with open("./embeddings/{}/bin/{}_{:05d}.pic".format(folder, batch, j), 'wb') as f:
        pc.dump(pr_tr, f)

    t = time.time()
    print("done {:05d} --- time: {}\t--- total: {}".format(j, t - temp_time, t - start_time))

# for i in pr_tr:
#     print(i, len(pr_tr[i]))

senteces from 677000 to 850000
done 00677 --- time: 134.54737520217896	--- total: 134.5488564968109
done 00678 --- time: 140.48352146148682	--- total: 275.0456817150116
done 00679 --- time: 133.68382096290588	--- total: 408.7411742210388
done 00680 --- time: 139.27910208702087	--- total: 548.0298917293549
done 00681 --- time: 140.92770624160767	--- total: 688.9688475131989
done 00682 --- time: 142.90595841407776	--- total: 831.8858373165131
done 00683 --- time: 141.86153626441956	--- total: 973.7582578659058
done 00684 --- time: 129.64669275283813	--- total: 1103.4155197143555
done 00685 --- time: 132.05880331993103	--- total: 1235.484465122223
done 00686 --- time: 140.3720462322235	--- total: 1375.8647136688232
done 00687 --- time: 140.94010138511658	--- total: 1516.8167872428894
done 00688 --- time: 145.92966675758362	--- total: 1662.7566578388214
done 00689 --- time: 144.0590853691101	--- total: 1806.8263409137726
done 00690 --- time: 137.4750874042511	--- total: 1944.312479019165
d

#aggregation of embeddings



In [ ]:
folder = "bert-base-multilingual-cased-liter"

##list of files

In [ ]:
files = os.listdir("./embeddings/{}/bin".format(folder))
cut_files = []
for i in files:
    if i.split('.')[-1] == 'pic':
        cut_files.append(i)

##get count

In [ ]:
%%time
from_files = {}
all_count = {}

for name in cut_files:
    with open("./embeddings/{}/bin/{}".format(folder, name), 'rb') as f:
        data = pc.load(f)
    for i in data.keys():
        if i not in all_count:
            all_count[i] = len(data[i])
            from_files[i] = [name]
        else:
            all_count[i] += len(data[i])
            from_files[i].append(name)
    # print(name)


CPU times: user 15min 43s, sys: 39.2 s, total: 16min 22s
Wall time: 35min 31s


In [ ]:
out = list(zip(all_count.keys(), list(all_count.values())))
out.sort(key = lambda i: i[1], reverse=True)
out[:20]

In [ ]:
all_words = []
for i in kb:
    if all_count[i] > 10:
        all_words.append(i)
len(all_words)

37553

In [ ]:
def check_existing_emb(words):
    d = dict(out)
    fl = True
    for i in words:
        if i not in d:
            print("{} not in dictionary".format(i))
            fl = False
    return fl

def get_all_embeddings(words):
    all_files = []
    embeddings = {}
    for w in words:
        embeddings[w] = []

    for i in words:
        all_files += from_files[i]

    for name in set(all_files):
        with open("./embeddings/{}/bin/{}".format(folder, name), 'rb') as f:
            data = pc.load(f)
        for w in words:
            if w in data:
                embeddings[w] += data[w]
    return embeddings

def sum_embeddings(embeddings):
    res = {}
    res_emb = {}
    for i in embeddings:
        res[i] = torch.empty(len(embeddings[i]), embeddings[i][0].size(0))
        for j in range(len(embeddings[i])):
            res[i][j] = embeddings[i][j]
        res_emb[i] = res[i].mean(dim = 0)
    return res_emb

In [ ]:
batch = 5000
start_time = time.time()

ftom_b = 0
to_b = 10

print("words from {} to {}, max batchs = {}".format(ftom_b * batch, to_b * batch, len(all_words)//batch))

all_emb = {}
for b in range(ftom_b, to_b):
    t_time = time.time()
    words = all_words[b*batch:(b+1)*batch]
    emb = get_all_embeddings(words)
    res_emb = sum_embeddings(emb)
    
    with open("./embeddings/{}/ready/{}_{:05d}.pic".format(folder,batch, b), 'wb') as f:
        pc.dump(res_emb, f)

    print("done: {} --- time: {}\t--- total: {}".format(b,time.time() - t_time, time.time() - st_time))

#building dictionary


In [ ]:
folder = "bert-base-multilingual-cased-liter"

In [ ]:
files = os.listdir("./embeddings/{}/ready".format(folder))
res = dict()
for i in files:
    with open("./embeddings/{}/ready/{}".format(folder,i), "rb") as f:
        temp = pc.load(f)
        # print(type(temp))
        res.update(temp)

In [ ]:
with open("./embeddings/{}/dict_{}.pic".format(folder, folder), "bw") as f:
    pc.dump(res, f)

In [ ]:
with open("./embeddings/{}/dict_{}.json".format(folder, folder), 'w', encoding="utf-8") as f:
    json.dump(lista_items, f, indent=4)